Parameters to be tuned for XGBoost:
1. n_estimators (default = 100): The n_estimators parameter defines the size of the forest of decision trees being run
2. max_depth (default = 3): The max_depth parameter defines the maximum depth of each decision tree
3. reg_lambda (default = 1): The lambda parameter represents the L2 norm regularization of weights
4. learning_rate/eta (default = 0.1): The learning rate parameter, also known as eta, is the rate that each step size of decision trees are changed to prevent overfitting
5. missing (default = None (node to the right)): The missing parameter determines which value within the data will take the default path within the decision trees when encountered. 
6. objective (default = 'binary:logistic'): The objective defines the learning objective and its corresponding error loss function.

In [18]:
# Accuracies and Error Rates for the Default XGBoost Parameters
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file
from xgboost import XGBClassifier

X_train, y_train = load_svmlight_file("a9a.txt")
X_test, y_test = load_svmlight_file("a9a.t")

model = XGBClassifier()
print(model.get_params())

#Training
model = model.fit(X_train, y_train)
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]

#Training Error
accuracy = accuracy_score(y_train, predictions)
print("Training Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Training Error: %.2f%%" % (error * 100.0))

#Testing
model = model.fit(X_test, y_test)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

#Testing Error
accuracy = accuracy_score(y_test, predictions)
print("Testing Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Testing Error: %.2f%%" % (error * 100.0))

#Training Cross-Val
cross = cross_val_score(model, X_train, y_train, cv=7)

error = 0
accuracy = 0
counter = 0
cross = np.array(cross)
for i in range(len(cross)):
    print("Cross Val. (Training, 7 folds) Accuracy: %.2f%%" % (cross[i]*100))
    accuracy += cross[i]
    cross[i] = 1-cross[i]
    print("Cross Val. (Training, 7 folds) Error: %.2f%%" % (cross[i]*100))
    error += cross[i]
    counter += 1
accuracy = accuracy/counter
error = error/counter
print("Cross Val. Avg. (Training, 7 folds) Accuracy: %.2f%%" % (accuracy*100))
print("Cross Val. Avg. (Training, 7 folds) Error: %.2f%%" % (error*100))

#Testing Cross-Val
cross = cross_val_score(model, X_test, y_test, cv=7)

error = 0
accuracy = 0
counter = 0
cross = np.array(cross)
for i in range(len(cross)):
    print("Cross Val. (Testing, 7 folds) Accuracy: %.2f%%" % (cross[i]*100))
    accuracy += cross[i]
    cross[i] = 1-cross[i]
    print("Cross Val. (Testing, 7 folds) Error: %.2f%%" % (cross[i]*100))
    error += cross[i]
    counter += 1
accuracy = accuracy/counter    
error = error/counter
print("Cross Val. Avg. (Testing, 7 folds) Accuracy: %.2f%%" % (accuracy*100))
print("Cross Val. Avg. (Testing, 7 folds) Error: %.2f%%" % (error*100))


{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': None, 'n_estimators': 100, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}
Training Accuracy: 84.87%
Training Error: 15.13%
Testing Accuracy: 85.34%
Testing Error: 14.66%
Cross Val. (Training, 7 folds) Accuracy: 84.37%
Cross Val. (Training, 7 folds) Error: 15.63%
Cross Val. (Training, 7 folds) Accuracy: 84.63%
Cross Val. (Training, 7 folds) Error: 15.37%
Cross Val. (Training, 7 folds) Accuracy: 83.83%
Cross Val. (Training, 7 folds) Error: 16.17%
Cross Val. (Training, 7 folds) Accuracy: 84.82%
Cross Val. (Training, 7 folds) Error: 15.18%
Cross Val. (Training, 7 folds) Accuracy: 84.54%
Cross Val. (Training, 7 folds) Error: 15.46%
Cross Val. (Training, 7 folds) Accur

In [2]:
# Grid Search Tuning
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file
from xgboost import XGBClassifier
from xgboost import XGBRegressor

X_train, y_train = load_svmlight_file("a9a.txt")
X_test, y_test = load_svmlight_file("a9a.t")

# 1. n_estimators
xgb_n_estimators = [int(x) for x in np.linspace(50, 60, 11)]
print("Estimators:", xgb_n_estimators)

# 2. max_depth
xgb_max_depth = [int(x) for x in np.linspace(3, 10, 9)]
print("Max Depths:", xgb_max_depth)

# 3. lambda
xgb_lambda = [0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
print("Lambdas:", xgb_lambda)

# 4. eta
xgb_learning_rate = [0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
print("Learning Rates:", xgb_learning_rate)

# 5. missing
xgb_missing = [None, 0]
print("Missings:", xgb_missing)

# 6. objective
xgb_objective = ['binary:logistic', 'binary:hinge']
print("Objectives:", xgb_objective)

optimized = []
for i in range(0, 3):
    if i==0:
        para_g = {'n_estimators': xgb_n_estimators, 'max_depth': xgb_max_depth}
        xgb_search = GridSearchCV(estimator=XGBClassifier(), param_grid = para_g)
        xgb_search.fit(X_train, y_train)
        best = xgb_search.best_params_
        print(best)
        optimized.append(best['n_estimators'])
        optimized.append(best['max_depth'])
    elif i==1:
        para_g = {'learning_rate': xgb_learning_rate, 'reg_lambda': xgb_lambda}
        xgb_search = GridSearchCV(estimator=XGBClassifier(), param_grid = para_g)
        xgb_search.fit(X_train, y_train)
        best = xgb_search.best_params_
        print(best)
        optimized.append(best['learning_rate'])
        optimized.append(best['reg_lambda'])
    else:
        para_g = {'objective': xgb_objective, 'missing': xgb_missing}
        xgb_search = GridSearchCV(estimator=XGBClassifier(), param_grid = para_g)
        xgb_search.fit(X_train, y_train)
        best = xgb_search.best_params_
        print(best)
        optimized.append(best['objective'])
        optimized.append(best['missing'])
        
model = XGBClassifier(n_estimators=optimized[0],
                      max_depth=optimized[1],
                      learning_rate=optimized[2],
                      reg_lambda=optimized[3],
                      objective=optimized[4],
                      missing=optimized[5])

#Training
model = model.fit(X_train, y_train)
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]

#Training Error
accuracy = accuracy_score(y_train, predictions)
print("Training Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Training Error: %.2f%%" % (error * 100.0))

#Testing
model = model.fit(X_test, y_test)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

#Testing Error
accuracy = accuracy_score(y_test, predictions)
print("Testing Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Testing Error: %.2f%%" % (error * 100.0))

Estimators: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
Max Depths: [3, 3, 4, 5, 6, 7, 8, 9, 10]
Lambdas: [0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
Learning Rates: [0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
Missings: [None, 0]
Objectives: ['binary:logistic', 'binary:hinge']
{'max_depth': 8, 'n_estimators': 55}
{'learning_rate': 0.225, 'reg_lambda': 0.05}
{'missing': None, 'objective': 'binary:logistic'}
Training Accuracy: 87.87%
Training Error: 12.13%
Testing Accuracy: 89.34%
Testing Error: 10.66%


In [25]:
# Manual Tuning
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file
from xgboost import XGBClassifier
from xgboost import XGBRegressor

X_train, y_train = load_svmlight_file("a9a.txt")
X_test, y_test = load_svmlight_file("a9a.t")

optimized = [120, 35, 0.225, 0.05, 'binary:logistic', None]
model = XGBClassifier(n_estimators=optimized[0],
                      max_depth=optimized[1],
                      learning_rate=optimized[2],
                      reg_lambda=optimized[3],
                      missing=optimized[5],
                      objective=optimized[4])
#Training
model = model.fit(X_train, y_train)
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]

#Training Error
accuracy = accuracy_score(y_train, predictions)
print("Training Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Training Error: %.2f%%" % (error * 100.0))

#Testing
model = model.fit(X_test, y_test)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

#Testing Error
accuracy = accuracy_score(y_test, predictions)
print("Testing Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Testing Error: %.2f%%" % (error * 100.0))

Training Accuracy: 95.60%
Training Error: 4.40%
Testing Accuracy: 96.80%
Testing Error: 3.20%


Final Parameters for XGBoost:
1. n_estimators (final = 110)
2. max_depth (final = 30)
3. reg_lambda (final = 0.05)
4. learning_rate/eta (final = 0.225)
5. missing (final = None))
6. objective (final = binary:logistic)

| n_estimators | max_depth | reg_lambda | learning_rate | missing | objective | Training Accuracy | Training Error | Testing Accuracy | Testing Error |
|---:|:-------------|:-----------|:------|:------|:------|:------|:------|:------|:------|
| 100 | 3 | 0.1 | 1 | None | 'binary:logistic' | 84.87% | 15.13% | 85.34% | 14.66% |
| 55 | 8 | 0.05 | 0.225 | None | 'binary:logistic' | 87.87% | 12.13% | 89.34% | 10.66% |
| 60 | 8 | 0.05 | 0.225 | None | 'binary:logistic' | 88.11% | 11.89% | 89.55% | 10.45% |
| 60 | 10 | 0.05 | 0.225 | None | 'binary:logistic' | 89.56% | 10.44% | 91.51% | 8.49% |
| 65 | 10 | 0.05 | 0.225 | None | 'binary:logistic' | 89.77% | 10.23% | 91.76% | 8.24% |
| 65 | 12 | 0.05 | 0.225 | None | 'binary:logistic' | 91.16% | 8.84% | 93.51% | 6.49% |
| 70 | 12 | 0.05 | 0.225 | None | 'binary:logistic' | 91.33% | 8.67% | 93.69% | 6.31% |
| 70 | 15 | 0.05 | 0.225 | None | 'binary:logistic' | 93.41% | 6.59% | 95.19% | 4.81% |
| 75 | 15 | 0.05 | 0.225 | None | 'binary:logistic' | 93.55% | 6.45% | 95.35% | 4.65% |
| 75 | 20 | 0.05 | 0.225 | None | 'binary:logistic' | 94.96% | 5.04% | 96.47% | 3.53% |
| 80 | 20 | 0.05 | 0.225 | None | 'binary:logistic' | 95.06% | 4.94% | 96.51% | 3.49% |
| 80 | 22 | 0.05 | 0.225 | None | 'binary:logistic' | 95.33% | 4.67% | 96.64% | 3.36% |
| 85 | 22 | 0.05 | 0.225 | None | 'binary:logistic' | 95.36% | 4.64% | 96.67% | 3.33% |
| 90 | 22 | 0.05 | 0.225 | None | 'binary:logistic' | 95.43% | 4.57% | 96.69% | 3.31% |
| 100 | 25 | 0.05 | 0.225 | None | 'binary:logistic' | 95.55% | 4.45% | 96.79% | 3.21% |
| 110 | 30 | 0.05 | 0.225 | None | 'binary:logistic' | 95.60% | 4.40% | 96.80% | 3.20% |
| 120 | 35 | 0.05 | 0.225 | None | 'binary:logistic' | 95.60% | 4.40% | 96.80% | 3.20% |

In [26]:
# Accuracies and Error Rates for the Final XGBoost Parameters
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file
from xgboost import XGBClassifier
from xgboost import XGBRegressor

X_train, y_train = load_svmlight_file("a9a.txt")
X_test, y_test = load_svmlight_file("a9a.t")

optimized = [110, 30, 0.225, 0.05, 'binary:logistic', None]
model = XGBClassifier(n_estimators=optimized[0],
                      max_depth=optimized[1],
                      learning_rate=optimized[2],
                      reg_lambda=optimized[3],
                      missing=optimized[5],
                      objective=optimized[4])
#Training
model = model.fit(X_train, y_train)
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]

#Training Error
accuracy = accuracy_score(y_train, predictions)
print("Training Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Training Error: %.2f%%" % (error * 100.0))

#Testing
model = model.fit(X_test, y_test)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

#Testing Error
accuracy = accuracy_score(y_test, predictions)
print("Testing Accuracy: %.2f%%" % (accuracy * 100.0))
error = 1-accuracy
print("Testing Error: %.2f%%" % (error * 100.0))

#Training Cross-Val
cross = cross_val_score(model, X_train, y_train, cv=7)

error = 0
accuracy = 0
counter = 0
cross = np.array(cross)
for i in range(len(cross)):
    print("Cross Val. (Training, 7 folds) Accuracy: %.2f%%" % (cross[i]*100))
    accuracy += cross[i]
    cross[i] = 1-cross[i]
    print("Cross Val. (Training, 7 folds) Error: %.2f%%" % (cross[i]*100))
    error += cross[i]
    counter += 1
accuracy = accuracy/counter
error = error/counter
print("Cross Val. Avg. (Training, 7 folds) Accuracy: %.2f%%" % (accuracy*100))
print("Cross Val. Avg. (Training, 7 folds) Error: %.2f%%" % (error*100))

#Testing Cross-Val
cross = cross_val_score(model, X_test, y_test, cv=7)

error = 0
accuracy = 0
counter = 0
cross = np.array(cross)
for i in range(len(cross)):
    print("Cross Val. (Testing, 7 folds) Accuracy: %.2f%%" % (cross[i]*100))
    accuracy += cross[i]
    cross[i] = 1-cross[i]
    print("Cross Val. (Testing, 7 folds) Error: %.2f%%" % (cross[i]*100))
    error += cross[i]
    counter += 1
accuracy = accuracy/counter    
error = error/counter
print("Cross Val. Avg. (Testing, 7 folds) Accuracy: %.2f%%" % (accuracy*100))
print("Cross Val. Avg. (Testing, 7 folds) Error: %.2f%%" % (error*100))

Training Accuracy: 95.60%
Training Error: 4.40%
Testing Accuracy: 96.80%
Testing Error: 3.20%
Cross Val. (Training, 7 folds) Accuracy: 82.29%
Cross Val. (Training, 7 folds) Error: 17.71%
Cross Val. (Training, 7 folds) Accuracy: 82.35%
Cross Val. (Training, 7 folds) Error: 17.65%
Cross Val. (Training, 7 folds) Accuracy: 81.60%
Cross Val. (Training, 7 folds) Error: 18.40%
Cross Val. (Training, 7 folds) Accuracy: 82.55%
Cross Val. (Training, 7 folds) Error: 17.45%
Cross Val. (Training, 7 folds) Accuracy: 83.75%
Cross Val. (Training, 7 folds) Error: 16.25%
Cross Val. (Training, 7 folds) Accuracy: 82.28%
Cross Val. (Training, 7 folds) Error: 17.72%
Cross Val. (Training, 7 folds) Accuracy: 82.65%
Cross Val. (Training, 7 folds) Error: 17.35%
Cross Val. Avg. (Training, 7 folds) Accuracy: 82.49%
Cross Val. Avg. (Training, 7 folds) Error: 17.51%
Cross Val. (Testing, 7 folds) Accuracy: 82.03%
Cross Val. (Testing, 7 folds) Error: 17.97%
Cross Val. (Testing, 7 folds) Accuracy: 82.63%
Cross Val. (Te